# Quickstart: Azure Blob Storage client library for Go

Azure Blob Storage is Microsoft's object storage solution for the cloud. Blob storage is optimized for storing massive amounts of unstructured data.

Follow steps to install the package and try out example code for basic tasks.

## Prerequisites

- Azure subscription - [create one for free](https://azure.microsoft.com/free/)
- Azure storage account - [create a storage account](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-create)

## Additional resources

- [Introduction to Azure Blob storage](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction)
- [API reference documentation](https://pkg.go.dev/github.com/Azure/azure-sdk-for-go/sdk/storage/azblob)
- [Library source code](https://github.com/Azure/azure-sdk-for-go/tree/main/sdk/storage/azblob)
- Samples

## Imports

First, import system packages and the Azure Blob Storage Go package.

In [32]:
package main

import (
  "context"
  "fmt"
  "strings"

  "github.com/Azure/azure-sdk-for-go/sdk/storage/azblob"
)

Next we'll need credentials to authenticate to the Storage service

In [33]:
accountName := os.Getenv("AZURE_STORAGE_ACCOUNT")
accountKey  := os.Getenv("AZURE_STORAGE_KEY")
serviceURL  := fmt.Sprintf("https://%s.blob.core.windows.net/", accountName)
ctx         := context.Background()

## Create a service client

The service client is the root client for accessing the storage account.

In [34]:
cred, _ := azblob.NewSharedKeyCredential(accountName, accountKey)
serviceClient, _ := azblob.NewServiceClientWithSharedKey(serviceURL, cred, nil)

## Create a container

To create a container, obtain a container client from the service client specifying the name of the container to create.  Then call the Create method on the container client.

In [35]:
containerName := "playground"

In [ ]:
container := serviceClient.NewContainerClient(containerName)
_, err := container.Create(ctx, nil)
if err != nil {
	fmt.Printf("Error: %s", err.Error())
}

Note that if the container already exists, the Create method will return an error.  Usually this is something that can be 

The following code illustrates how to check for and ignore specific errors using the [`StorageErrorCode`](https://pkg.go.dev/github.com/Azure/azure-sdk-for-go/sdk/storage/azblob#StorageErrorCode) in the Azure Storage Error response.

In [36]:
container := serviceClient.NewContainerClient(containerName)
_, err := container.Create(ctx, nil)
if err != nil {
	if strings.Contains(err.Error(), fmt.Sprintf("ErrorCode=%s", azblob.StorageErrorCodeContainerAlreadyExists)) {
		// Container exists -- this is fine
	} else {
		fmt.Printf("Error: %s", err.Error())
	}
}

## Upload a blob to a container

To store data in a byte buffer to a blob, obtain a BlockBlob client from the container client, specifying the name of the blob.

The data is passed to the Upload method as a [`ReadSeekCloser`](https://pkg.go.dev/io#ReadSeekCloser). The `streaming` package in the Azure SDK for Go provides the `NopCloser` method to obtain a `ReadSeekCloser` from a `ReadSeeker`.

In [37]:
blobName := "blob.txt"

// blobData could come from a file, a network call, or elsewhere. Here we just use a static string.
blobData := []byte("Hello World")

// Get a blobClient for the blob
blockBlob := container.NewBlockBlobClient(blobName)

In [40]:
// Upload the blobData to the blob
_, err := blockBlob.Upload(ctx, streaming.NopCloser(bytes.NewReader(blobData)), /* options */ nil)

ERROR: type <github.com/Azure/azure-sdk-for-go/sdk/storage/azblob.BlockBlobClient>: inconsistent 5-th method signature:
	go/types.Type has receiver <var  github.com/Azure/azure-sdk-for-go/sdk/storage/azblob.BlockBlobClient> and 3 parameters: func(context.Context, io.ReadSeekCloser, *github.com/Azure/azure-sdk-for-go/sdk/storage/azblob.UploadBlockBlobOptions) (github.com/Azure/azure-sdk-for-go/sdk/storage/azblob.BlockBlobUploadResponse, error)
	reflect.Type has 3 parameters: func(azblob.BlockBlobClient, context.Context, *azblob.DownloadBlobOptions) (*azblob.DownloadResponse, error)